# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import numpy as np
import pandas as pd
from sql_queries import *
import json

In [50]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()


In [54]:
conn.close()

In [2]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [ ]:
song_files = get_files('song_data')
song_files



In [48]:
df = pd.DataFrame()
for file in song_files:
    tmp_df = pd.read_json(file, lines=True)
    df = df.append(tmp_df)

df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0
0,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969
0,1,ARKRRTF1187B9984DA,NaN,NaN,,Sonora Santanera,SOXVLOJ12AB0189215,Amor De Cabaret,177.47546,0
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
0,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [ ]:
song_col = df[['song_id','title','artist_id','year','duration']].copy()
song_data = list(song_col.values)
song_data


In [61]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()


In [59]:
conn.close()

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [62]:
for data in song_data:
    cur.execute(song_table_insert, data)
    conn.commit()

In [ ]:
cur.execute("select * from song_table limit 10")
cur.fetchall()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [16]:
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0
0,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969
0,1,ARKRRTF1187B9984DA,NaN,NaN,,Sonora Santanera,SOXVLOJ12AB0189215,Amor De Cabaret,177.47546,0
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
0,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007


In [ ]:
art_col = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values.tolist()
artist_data = art_col
artist_data

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [63]:
for data in artist_data:
    cur.execute(artist_table_insert, data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [65]:
log_files = get_files('log_data')
log_files

['e:\\Udacity\\UdacityProjects\\postgres_datamodeling\\log_data\\2018\\11\\2018-11-01-events.json',
 'e:\\Udacity\\UdacityProjects\\postgres_datamodeling\\log_data\\2018\\11\\2018-11-02-events.json',
 'e:\\Udacity\\UdacityProjects\\postgres_datamodeling\\log_data\\2018\\11\\2018-11-03-events.json',
 'e:\\Udacity\\UdacityProjects\\postgres_datamodeling\\log_data\\2018\\11\\2018-11-04-events.json',
 'e:\\Udacity\\UdacityProjects\\postgres_datamodeling\\log_data\\2018\\11\\2018-11-05-events.json',
 'e:\\Udacity\\UdacityProjects\\postgres_datamodeling\\log_data\\2018\\11\\2018-11-06-events.json',
 'e:\\Udacity\\UdacityProjects\\postgres_datamodeling\\log_data\\2018\\11\\2018-11-07-events.json',
 'e:\\Udacity\\UdacityProjects\\postgres_datamodeling\\log_data\\2018\\11\\2018-11-08-events.json',
 'e:\\Udacity\\UdacityProjects\\postgres_datamodeling\\log_data\\2018\\11\\2018-11-09-events.json',
 'e:\\Udacity\\UdacityProjects\\postgres_datamodeling\\log_data\\2018\\11\\2018-11-10-events.json',


In [39]:
# filepath = 'e:\\Udacity\\UdacityProjects\\postgres_datamodeling\\log_data\\2018\\11\\2018-11-01-events.json'

In [66]:
df = pd.DataFrame()
for file in log_files:
    tmp_df = pd.read_json(file, lines=True)
    df = df.append(tmp_df)
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1.540919e+12,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1.540345e+12,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [67]:
# Convert ts to datetime. from milliseconds
df['ts'] = pd.to_datetime(df['ts'], unit='ms') # changes timstamp from ms to datetime timestamp
# df.rename(columns = {'timestamp':'ts'}, inplace = True) # rename ts column to timestamp


In [68]:
# Filter Dataframe based on NextSong being in page column
df = df[df.page.isin(['NextSong'])]
df.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,2018-11-01 21:01:46.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,2018-11-01 21:05:52.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Quem Quiser Encontrar O Amor,200,2018-11-01 21:08:16.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [171]:
import datetime as dt

In [69]:
# Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
df['start_time'] = df['ts'].dt.strftime('%H:%M:%S.%f')

start_time = df['start_time']
hour = df['ts'].dt.hour
day = df['ts'].dt.day  
weekofyear = df['ts'].dt.strftime('%W')
month = df['ts'].dt.month
year = df['ts'].dt.year
weekday = df['ts'].dt.dayofweek # The day of the week with Monday=0, Sunday=6.


# time_data
time_data = [start_time, hour, day, weekofyear, month, year, weekday]

column_labels = ('start_time','hour','day','weekofyear','month','year','weekday')


In [10]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [19]:
conn.close()

In [70]:
column_data = {column_labels[0]: pd.Series(time_data[0]),
                column_labels[1]: pd.Series(time_data[1]),
                column_labels[2]: pd.Series(time_data[2]),
                column_labels[3]: pd.Series(time_data[3]),
                column_labels[4]: pd.Series(time_data[4]),
                column_labels[5]: pd.Series(time_data[5]),
                column_labels[6]: pd.Series(time_data[6])}
time_df = pd.DataFrame(column_data)
time_df.head()

,start_time,hour,day,weekofyear,month,year,weekday
2,21:01:46.796000,21,1,44,11,2018,3
4,21:05:52.796000,21,1,44,11,2018,3
5,21:08:16.796000,21,1,44,11,2018,3
6,21:11:13.796000,21,1,44,11,2018,3
7,21:17:33.796000,21,1,44,11,2018,3


In [25]:
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,start_time
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,2018-11-01 21:01:46.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,21:01:46.796000
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,2018-11-01 21:05:52.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,21:05:52.796000


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [71]:

for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()
   

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [72]:
user_df = df[['userId','firstName','lastName','gender','level']]
user_df

,userId,firstName,lastName,gender,level
2,8,Kaylee,Summers,F,free
4,8,Kaylee,Summers,F,free
5,8,Kaylee,Summers,F,free
6,8,Kaylee,Summers,F,free
7,8,Kaylee,Summers,F,free
...,...,...,...,...,...
382,16,Rylan,George,M,paid
383,16,Rylan,George,M,paid
384,16,Rylan,George,M,paid
385,16,Rylan,George,M,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [73]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [33]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
# conn.close()

In [22]:
cur.execute(song_select)
cur.fetchone()

SyntaxError: syntax error at or near "%"
LINE 5:                     where a.artist_name = %s 
                                                  ^


In [88]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,start_time
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,2018-11-01 21:01:46.796,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,8,21:01:46.796000
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,2018-11-01 21:05:52.796,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,8,21:05:52.796000
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Quem Quiser Encontrar O Amor,200,2018-11-01 21:08:16.796,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,8,21:08:16.796000
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Eriatarka,200,2018-11-01 21:11:13.796,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,8,21:11:13.796000
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Becoming Insane,200,2018-11-01 21:17:33.796,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,8,21:17:33.796000


In [87]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()



In [32]:
conn.close()

In [89]:
# art_col = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]
# ad = art_col.values.tolist()
# artist_data = ad
# artist_data
df['userAgent'] = df['userAgent'].str.strip('""')
# df['userId'].replace(0,1)
df.head()



,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,start_time
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,2018-11-01 21:01:46.796,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,8,21:01:46.796000
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,2018-11-01 21:05:52.796,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,8,21:05:52.796000
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Quem Quiser Encontrar O Amor,200,2018-11-01 21:08:16.796,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,8,21:08:16.796000
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Eriatarka,200,2018-11-01 21:11:13.796,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,8,21:11:13.796000
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Becoming Insane,200,2018-11-01 21:17:33.796,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,8,21:17:33.796000


In [30]:
time_df['start_time'].head()

2    21:01:46.796000
4    21:05:52.796000
5    21:08:16.796000
6    21:11:13.796000
7    21:17:33.796000
Name: start_time, dtype: object

In [90]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchall()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # # insert songplay record
    # data = df[['ts','userId', 'level', 'sessionId', ]]
    songplay_data = (row.start_time, str(row.userId), row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

UndefinedColumn: column "timestamp" of relation "songplay_table" does not exist
LINE 1:  INSERT INTO songplay_table (timestamp, userId, level, songi...
                                     ^


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [92]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.